In [1]:
from common import *

In [2]:
def submit(y_, name):
    from pathlib import Path
    from decimal import Decimal
    '''Params - y_: 1-D `ndarray` of the final actual values to submit, name: `str` of the file name
    Saves `y_` in the given name csv file in data/submit. If `y_` is a `Series`, the index must be the desired index.'''
    ypred = (pd.Series(y_).rename('y').astype(str).map(Decimal) / 10000).astype(str)
    ypred.index.name = 'Index'
    path = Path(top_dir + 'data/submit/' + name + '.csv')
    assert not path.is_file(), 'Please manually delete old submission with the same name first'
    ypred.to_csv(path, header=True)

In [9]:
if __name__ == '__main__':
    zero = np.zeros(len(dg.index))
    submit(zero, 'zero')
    stock_mean = dg.Stock.map(df.groupby('Stock').y.mean()).fillna(0)
    submit(stock_mean, 'stock_mean')